In [222]:
# les bibliotheques
from scipy.stats import norm
from math import *


## Pricing des options vanilles de type europeen avec BS

### 1-Valeur du call et put

In [223]:
# les inputs

s = 100 # taux de change à t=0
k = 100  # le strike(ou prix d'exercice)
t = 0.5 # la date d'echeance  (=nbre de jour/360)
rf = 0 # taux d'interet etranger
r = 0.05  # taux d'interet domestique
sigma = 0.35 # volatilité du taux de change

# Output: valeur du call ou put          
def bs_price_call(s,k,t,r,rf,sigma):
    d1 = (log(s/k) + (r - rf + 0.5*sigma**2)*t) / (sigma*sqrt(t))  
    d2 = d1-sigma*sqrt(t)
    price_call = s * exp(-rf*t) * norm.cdf(d1) - (k*exp(-r*t)*norm.cdf(d2))
    return price_call
def bs_price_put(s,k,t,r,rf,sigma):
    d1= (log(s/k)+(r-rf+0.5*sigma**2)*t) / (sigma*sqrt(t))  
    d2 = d1-sigma*sqrt(t)
    price_put = k*exp(-r*t)*norm.cdf(-d2)-(s*exp(-rf*t)*norm.cdf(-d1)) 
    return price_put
        

In [224]:
bs_price_call(s,k,t,r,rf,sigma)

11.010433748432774

### 2-Les lettres grecques

In [225]:
def delta_call(s,k,t,r,rf,sigma):
    d1= (log(s/k)+(r-rf+0.5*sigma**2)*t) / (sigma*sqrt(t))  
    return exp(-r*t)*norm.cdf(d1)
def delta_put(s,k,t,r,rf,sigma):
    d1= (log(s/k)+(r-rf+0.5*sigma**2)*t) / (sigma*sqrt(t))  
    return exp(-r*t) * (norm.cdf(d1)-1)
def gamma_call_put(s,k,t,r,rf,sigma):
    d1= (log(s/k)+(r-rf+0.5*sigma**2)*t) / (sigma*sqrt(t))  
    return norm.pdf(d1)*exp(-rf*t)/s*sigma*sqrt(t)
def theta_call(s,k,t,r,rf,sigma):
    d1= (log(s/k)+(r-rf+0.5*sigma**2)*t) / (sigma*sqrt(t))
    d2 = d1-sigma*sqrt(t)
    a = -s*norm.pdf(d1)*sigma*exp(-rf*t) /(2*sqrt(t))
    b = rf*s*norm.cdf(d1)*exp(-rf*t) 
    c = r*k*exp(-r*t)*norm.cdf(d2)
    return a+b-c
def theta_put(s,k,t,r,rf,sigma):
    d1= (log(s/k)+(r-rf+0.5*sigma**2)*t) / (sigma*sqrt(t))  
    d2 = d1-sigma*sqrt(t)
    a = -s*norm.pdf(d1)*sigma*exp(-rf*t) /(2*sqrt(t))
    b = rf*s*norm.cdf(-d1)*exp(-rf*t) 
    c = r*k*exp(-r*t)*norm.cdf(-d2)
    return a-b+c   
def vega_call_put(s,k,t,r,rf,sigma):
    d1= (log(s/k)+(r-rf+0.5*sigma**2)*t) / (sigma*sqrt(t))  
    return s*sqrt(t)*norm.pdf(d1)*exp(-rf*t)
def vanna_call_put(s,k,t,r,rf,sigma):
    return 
def volga_call_put(s,k,t,r,rf,sigma):
    return 


In [226]:
call = bs_price_call(s,k,t,r,rf,sigma)
print("prix du call = ", call)
print( "delta = " ,delta_call(s,k,t,r,rf,sigma))
print( "gamma = " , gamma_call_put(s,k,t,r,rf,sigma))
print( "theta = " , theta_call(s,k,t,r,rf,sigma))
print( "vega = " , vega_call_put(s,k,t,r,rf,sigma))

prix du call =  11.010433748432774
delta =  0.5743761929305299
gamma =  0.0009627057727444755
theta =  -12.021119017427761
vega =  27.50587922127073


In [227]:
put = bs_price_put(s,k,t,r,rf,sigma)
print("prix du put = ", put)
print( "delta = " ,delta_put(s,k,t,r,rf,sigma))
print( "gamma = " , gamma_call_put(s,k,t,r,rf,sigma))
print( "theta = " , theta_put(s,k,t,r,rf,sigma))
print( "vega = " , vega_call_put(s,k,t,r,rf,sigma))

prix du put =  8.541424951266038
delta =  -0.40093371909780273
gamma =  0.0009627057727444755
theta =  -7.1445694572860985
vega =  27.50587922127073


## Volatilité implicite: Methode de Newton-Raphson

In [228]:
def NewtRaph(target_value, s, k, t, r, rf):
    MAX_ITERATIONS = 100
    PRECISION = 1.0e-5
    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_price_call(s,k,t,r,rf,sigma)
        vega = vega_call_put(s,k,t,r,rf,sigma)
        diff = target_value - price  # our root
        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega 
    return sigma 

In [229]:
s = 100
k = 100
t = 0.5 # 11
r = 0.05 #0.01
vol = 0.35

V_market = bs_price_call(s,k,t,r,rf,sigma)
implied_vol = NewtRaph(V_market, s, k, t, r, rf)

print ('Implied vol: %.2f%%' % (implied_vol * 100))
print ('Market price = %.2f' % V_market)
print ('Model price = %.2f' % bs_price_call(s, k, t, r,rf, implied_vol))

Implied vol: 35.00%
Market price = 11.01
Model price = 11.01


## Volatilité locale : Modele de Dupire


In [232]:
def deriv1():
    h = 0.00001
    sigma = 0.01 # problem here: which value for initialization ?
    return ( bs_price_call(s,k,t+h,r,rf,sigma)-bs_price_call(s,k,t-h,r,rf,sigma) ) / (2*h)
def deriv2():
    h = 0.00001
    sigma = 0.01
    return ( bs_price_call(s,k-h,t,r,rf,sigma) - 2*bs_price_call(s,k,t,r,rf,sigma)+ bs_price_call(s,k+h,t,r,rf,sigma) ) / (h**2)
    
def dupire(s,k,t,r,rf):
    local = sqrt( (deriv1() ) / ( 0.5*(k**2)*deriv2() ) )
    return local

In [233]:
dupire(s,k,t,r,rf)

0.9901280561480059

## Volatilité stochastique

### 1-Modele GARCH

### 2-Modele de Helson